In [115]:
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import ipywidgets as widgets
import json
import csv
import tweepy
import re
%matplotlib inline

In [116]:
base_url = 'http://spectrum-prod-elb.digitalsmiths.net/sd/spectrum/cores/catalog-spectrum-assets/search?q=*:*&join={!inner core=catalog-spectrum-offers left=id right=workId fq=divisionCode:SMN as=offers}&join={!inner core=rotten-tomatoes-charter left=guid right=guid as=rt}&join={!left core=assets-cs left=guid right=guid as=cs}&fq=objectType:Movie&offers.fq=startTime:[* TO NOW] endTime:[NOW TO *]'

In [117]:
genre_list = ['Reality','Drama','Adventure','Animated','Comedy','Documentary','Fantasy','Action','Anime','Sitcom','Crimedrama','Entertainment','Sciencefiction','Special','Thriller','Romance','Cooking','Crime','Educational','Mystery','House/garden','Talk','Comedydrama','Travel','Horror','History','Soap','News','Interview','Animals','Science','Music','Homeimprovement','How-to','Medical','Nature','Auto','Paranormal','Gameshow','Western','Historicaldrama','Law','Outdoors','Romanticcomedy','Holiday','Religious','Miniseries','War','Politics','Biography','Fashion','Publicaffairs','Anthology','Docudrama','Health','Art','Standup','Newsmagazine','Martialarts','Parenting','Gay/lesbian','Military','Musical','Fishing','Dance','Exercise','Community','Technology','Football','Prowrestling','Autoracing','Environment','Collectibles','Auction','Darkcomedy','Gaming','Sportstalk','Musicalcomedy','Actionsports','Aviation','Basketball','Consumer','Worldhistory','Selfimprovement','Mixedmartialarts','Americanhistory','Baseball','Performingarts','Soccer','Arts/crafts','Agriculture','Golf','Computers']

In [118]:
#Generates dropdown that references the Genre list for the options
#then returns a Tivo query genre parameter based on the selection

drop_down = widgets.Dropdown(options=genre_list,
                                                    
                                value="Adventure",
                                                      #sets name of dropdown
                                description='Genre',
                                disabled=False)

def set_director(genre_selection): 
    
    global genre_param
    genre_param = f'&fq=genres.name:{genre_selection}'

    return genre_param
    
widgets.interact(set_director, genre_selection=drop_down);

interactive(children=(Dropdown(description='Genre', index=2, options=('Reality', 'Drama', 'Adventure', 'Animat…

In [5]:
#Generates slider of 0-100. the selection returns a list E.G [8, 15] 
#This list is then referenced to create the desired query param
slider = widgets.IntRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1,
    description='RT Score:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def set_score_range(score_range):
    
    global score_param
    score_param = f'&rt.fq=ratings.audience_score:[{str(score_range[0])} TO {str(score_range[1])}]'
    return score_param



widgets.interact(set_score_range, score_range=slider);

interactive(children=(IntRangeSlider(value=(0, 100), continuous_update=False, description='RT Score:'), Output…

In [6]:
#Generates slider of 0-50. the selection returns a list E.G [8, 15] 
#This list is then referenced to create the desired query param

slider = widgets.IntRangeSlider(
    value=[0, 50],
    min=0,
    max=50,
    step=1,
    description='Target age:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def set_age_range(age_range):
    
    global age_param
    age_param = f'&cs.fq=metadata.review.slider.target_age:[{age_range[0]} TO {age_range[1]}]'
    return age_param



widgets.interact(set_age_range, age_range=slider);

interactive(children=(IntRangeSlider(value=(0, 50), continuous_update=False, description='Target age:', max=50…

In [119]:
#concatenates the final query 

tail_url = '&fl=cs.metadata.review.slider.on_age,id,title,releaseDate,releaseYear,genres,descriptors,rt.ratings.audience_score,rt.ratings.critics_score,offers.providerName,offers.startTime,offers.endTime'
query_url = base_url+genre_param+score_param+age_param+tail_url
query_url

'http://spectrum-prod-elb.digitalsmiths.net/sd/spectrum/cores/catalog-spectrum-assets/search?q=*:*&join={!inner core=catalog-spectrum-offers left=id right=workId fq=divisionCode:SMN as=offers}&join={!inner core=rotten-tomatoes-charter left=guid right=guid as=rt}&join={!left core=assets-cs left=guid right=guid as=cs}&fq=objectType:Movie&offers.fq=startTime:[* TO NOW] endTime:[NOW TO *]&fq=genres.name:Thriller&rt.fq=ratings.audience_score:[79 TO 100]&cs.fq=metadata.review.slider.target_age:[0 TO 50]&fl=cs.metadata.review.slider.on_age,id,title,releaseDate,releaseYear,genres,descriptors,rt.ratings.audience_score,rt.ratings.critics_score,offers.providerName,offers.startTime,offers.endTime'

In [120]:
#makes the actual API with the query 

response = requests.get(query_url).json()

In [121]:
#Creates Pandas data frame to hold the results

results = {}
results_df = pd.DataFrame(results)

results_df['TMS_ID'] = ""
results_df['title'] = ""
results_df['release_year'] = ""
results_df['genre'] = ""
results_df['audience_score'] = ""
results_df['critics_score'] = ""
results_df['start_time'] = ""
results_df['end_time'] = ""
results_df['provider_name'] = ""
results_df['OMDB_pop_score'] = ""
results_df['tweets'] = ""
results_df['combined_followers'] = ""
results_df

,TMS_ID,title,release_year,genre,audience_score,critics_score,start_time,end_time,provider_name,OMDB_pop_score,tweets,combined_followers


In [122]:
#iterates through the results and populatates the pandas dataframe columns with the cooresponding json value

for i in range(25):
    try:
        results_df.loc[i+1,'TMS_ID'] = response['hits'][i]['id']
        results_df.loc[i+1,'release_year'] = response['hits'][i]['releaseYear']
        genres = response['hits'][i]['genres']
        results_df.loc[i+1,'genre'] = [genres[j]['name']['en'] for j in range(len(genres))]
        results_df.loc[i+1,'audience_score'] = response['hits'][i]['rt'][0]['ratings']['audience_score']
        results_df.loc[i+1,'start_time']  = response['hits'][i]['offers'][0]['startTime']
        results_df.loc[i+1,'end_time'] = response['hits'][i]['offers'][0]['endTime']
        results_df.loc[i+1,'provider_name'] = response['hits'][i]['offers'][0]['providerName']
        results_df.loc[i+1,'title']= response['hits'][i]['title']['en']
        results_df.loc[i+1,'critics_score'] = response['hits'][i]['rt'][0]['ratings']['critics_score']                                                                               
        
    except:
        print(f'{i} has something weird')

In [123]:
results_df

,TMS_ID,title,release_year,genre,audience_score,critics_score,start_time,end_time,provider_name,OMDB_pop_score,tweets,combined_followers
1,MV001724250000,The Man Who Knew Too Much,1956,[Thriller],84,87,2019-03-21T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,NaN,NaN,NaN
2,MV001740600000,V for Vendetta,2006,"[Action, Thriller]",90,73,2016-07-14T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,NaN,NaN,NaN
3,MV000435430000,Seven,1995,"[Thriller, Crime drama]",95,82,2016-07-14T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,NaN,NaN,NaN
4,MV000468120000,Wake in Fright,1971,"[Drama, Thriller]",83,96,2017-05-15T04:00:00+00:00,2021-04-01T03:59:59+00:00,SHUDDER_SVOD,NaN,NaN,NaN
5,MV008879100000,Split,2016,"[Thriller, Horror]",79,77,2020-02-19T05:00:00+00:00,2023-01-01T04:59:59+00:00,IND_HD,NaN,NaN,NaN
6,MV004084190000,The Conjuring,2013,"[Horror, Thriller]",83,86,2017-06-26T04:00:00+00:00,2030-01-01T04:59:00+00:00,IND_HD_EXP,NaN,NaN,NaN
7,MV000004670000,Psycho,1960,"[Horror, Thriller]",95,96,2019-03-21T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,NaN,NaN,NaN
8,MV000171900000,Vertigo,1958,"[Thriller, Mystery]",93,94,2020-02-19T05:00:00+00:00,2022-10-31T03:59:59+00:00,IND_HD,NaN,NaN,NaN
9,MV013433790000,The Burnt Orange Heresy,2019,[Thriller],81,67,2020-08-25T04:00:00+00:00,2020-12-10T04:59:59+00:00,IND_HD,NaN,NaN,NaN
10,MV013636250000,Doctor Sleep: Director's Cut,2019,"[Horror, Thriller]",89,77,2020-09-21T10:00:00+00:00,2021-03-01T10:59:00+00:00,HBO_HD_EXP,NaN,NaN,NaN


In [124]:
#Bar graphs the results

titles = list(results_df['title'])
scores = list(results_df['audience_score'])

y = scores
x = titles
    #hovertext defaults to index (country name)
fig = go.Figure(data=[go.Bar(x=x, y=y)])
    # Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)' , marker_line_color='rgb(8,48,107)',
                      marker_line_width=1.5, opacity=0.6)
fig.update_layout(template= "plotly_dark",title_text='Rotten Tomatoes Score')
fig.show()

In [125]:
#Queries OMDB with each movie title and stores the popularity score in the pandas df

import warnings
warnings.filterwarnings('ignore')


for i in range(len(titles)):
    try:
        OMDB_url = f"https://api.themoviedb.org/3/search/movie?api_key=e07b872e9d8245d899b0f40ba2d998c4&query={titles[i]}"
        OMDB_response = requests.get(OMDB_url, verify=False).json()
        results_df.loc[i+1,'OMDB_pop_score'] = OMDB_response['results'][0]['popularity']
    except:
        pass
    
print('complete')

complete


In [126]:
results_df

,TMS_ID,title,release_year,genre,audience_score,critics_score,start_time,end_time,provider_name,OMDB_pop_score,tweets,combined_followers
1,MV001724250000,The Man Who Knew Too Much,1956,[Thriller],84,87,2019-03-21T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,9.235,NaN,NaN
2,MV001740600000,V for Vendetta,2006,"[Action, Thriller]",90,73,2016-07-14T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,33.732,NaN,NaN
3,MV000435430000,Seven,1995,"[Thriller, Crime drama]",95,82,2016-07-14T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,75.488,NaN,NaN
4,MV000468120000,Wake in Fright,1971,"[Drama, Thriller]",83,96,2017-05-15T04:00:00+00:00,2021-04-01T03:59:59+00:00,SHUDDER_SVOD,8.71,NaN,NaN
5,MV008879100000,Split,2016,"[Thriller, Horror]",79,77,2020-02-19T05:00:00+00:00,2023-01-01T04:59:59+00:00,IND_HD,53.567,NaN,NaN
6,MV004084190000,The Conjuring,2013,"[Horror, Thriller]",83,86,2017-06-26T04:00:00+00:00,2030-01-01T04:59:00+00:00,IND_HD_EXP,58.055,NaN,NaN
7,MV000004670000,Psycho,1960,"[Horror, Thriller]",95,96,2019-03-21T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,28.109,NaN,NaN
8,MV000171900000,Vertigo,1958,"[Thriller, Mystery]",93,94,2020-02-19T05:00:00+00:00,2022-10-31T03:59:59+00:00,IND_HD,20.895,NaN,NaN
9,MV013433790000,The Burnt Orange Heresy,2019,[Thriller],81,67,2020-08-25T04:00:00+00:00,2020-12-10T04:59:59+00:00,IND_HD,9.355,NaN,NaN
10,MV013636250000,Doctor Sleep: Director's Cut,2019,"[Horror, Thriller]",89,77,2020-09-21T10:00:00+00:00,2021-03-01T10:59:00+00:00,HBO_HD_EXP,NaN,NaN,NaN


In [127]:
#makes another pretty bar chart with the popularity score 

titles = list(results_df['title'])
popularity = list(results_df['OMDB_pop_score'])

y = popularity
x = titles
    #hovertext defaults to index (country name)
fig = go.Figure(data=[go.Bar(x=x, y=y)])
    # Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)' , marker_line_color='rgb(8,48,107)',
                      marker_line_width=1.5, opacity=0.6)
fig.update_layout(template= "plotly_dark",title_text='OMDB Popularity Score')
fig.show()

In [128]:
#Put Twitter Keys Here

consumer_key = 'ipj7n4K6kiYFnU9KmwGYeOa8j'
consumer_secret = 'aHQ9PWWwCVySezoIc78paOlFFRFYtpr7yoIjFtzAbNguBScRIc'
access_token = '1268967663738294272-ZpRjqVMCAId9nuiuvg734CvIkxGB0U'
access_token_secret = 'PX3TOcof5QSiqiwl6mMiORTBBqZrabNnhalZC5vmF434K'
    


In [129]:
#create authentication for accessing Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
api = tweepy.API(auth)
   
#Searches twitter for all tweets in the past 24 hours with the hashtag of the movie title 
#adds total tweet column and combined follwers of those tweeters for each movie

for i in range(len(results_df)):
    try:
        hashtag_phrase = results_df.loc[i+1, 'title']
        username = []
        followers = []
    except:
        print(f"input error with {hashtag_phrase}")
    
    try:
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                           lang="en", tweet_mode='extended').items(100):

            username.append(tweet.user.screen_name.encode('utf-8'))
            followers.append(tweet.user.followers_count)

            user_tweets = len(username)
            combined_followers = sum(followers)

        results_df.loc[i+1,"tweets"] = user_tweets
        results_df.loc[i+1,"combined_followers"] = combined_followers 
    
    except:
        print(f"API Error with {hashtag_phrase}")
                       
print('complete')    
 
    

complete


In [18]:
results_df.to_csv("Sherlock_Output.csv", index=False, header=True)

In [97]:
results_df

,TMS_ID,title,release_year,genre,audience_score,critics_score,start_time,end_time,provider_name,OMDB_pop_score,tweets,combined_followers,popularity
1,MV005848780000,Coming Home,2014,"[Drama, Romance]",85,89,2019-03-28T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,9.86,100,3634834,9.860
2,MV000413730000,Before Sunrise,1995,[Romance],93,100,2020-08-01T10:00:00+00:00,2021-02-01T10:59:00+00:00,HBO_HD_EXP,18.777,100,207989,18.777
3,MV001397560000,NaN,2003,"[Romance, Drama]",80,NaN,2020-12-04T05:00:00+00:00,2021-06-04T03:59:59+00:00,BOLLYWOOD,10.158,NaN,NaN,10.158
4,MV001445820000,NaN,2003,"[Comedy drama, Romance]",94,NaN,2020-11-20T05:00:00+00:00,2021-02-26T04:59:59+00:00,BOLLYWOOD,10.158,NaN,NaN,10.158
5,MV002100260000,Shelter,2007,"[Drama, Romance]",85,60,2020-08-26T04:00:00+00:00,2030-01-02T04:59:59+00:00,Here,13.619,100,9378330,13.619
6,MV002610690000,Don't Let Me Drown,2009,"[Drama, Romance]",81,NaN,2019-12-16T05:00:00+00:00,2030-01-01T04:59:59+00:00,UMC_SVOD,3.8,100,77575,3.800
7,MV001426420000,Eternal Sunshine of the Spotless Mind,2004,"[Romance, Science fiction]",94,93,2019-03-14T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,33.647,100,658261,33.647
8,MV012815800000,The Photograph,2020,"[Romance, Drama]",81,74,2020-12-06T05:00:00+00:00,2020-12-20T04:59:59+00:00,HBO_HD,26.274,100,2059432,26.274
9,MV000363120000,Groundhog Day,1993,"[Romantic comedy, Romance]",88,96,2016-07-21T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,18.005,100,256247,18.005
10,MV001941940000,Across the Universe,2007,"[Musical, Romance]",82,53,2020-07-10T11:01:00+00:00,2021-01-01T12:59:00+00:00,FXM_HD,8.625,100,378921,8.625


In [98]:
IP_consumption_df = pd.read_csv("IP_consumption.csv")

In [99]:
IP_consumption_df

,TMS_ID,vt_hours
0,EP000000211956,222.373056
1,EP000000211957,835.177778
2,EP000000211959,933.941667
3,EP000000211962,4999.984722
4,EP000000211963,4375.606944
...,...,...
61260,SH036864110000,0.279722
61261,SH036879510000,989.755278
61262,SH036902750000,44.620000
61263,SH036905140000,4.904444


In [100]:
total_results = pd.merge(results_df, IP_consumption_df, on="TMS_ID")

In [101]:
total_results

,TMS_ID,title,release_year,genre,audience_score,critics_score,start_time,end_time,provider_name,OMDB_pop_score,tweets,combined_followers,popularity,vt_hours
0,MV000413730000,Before Sunrise,1995,[Romance],93,100,2020-08-01T10:00:00+00:00,2021-02-01T10:59:00+00:00,HBO_HD_EXP,18.777,100,207989,18.777,125.269167
1,MV002100260000,Shelter,2007,"[Drama, Romance]",85,60,2020-08-26T04:00:00+00:00,2030-01-02T04:59:59+00:00,Here,13.619,100,9378330,13.619,8.735000
2,MV001426420000,Eternal Sunshine of the Spotless Mind,2004,"[Romance, Science fiction]",94,93,2019-03-14T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,33.647,100,658261,33.647,0.042222
3,MV000363120000,Groundhog Day,1993,"[Romantic comedy, Romance]",88,96,2016-07-21T04:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,18.005,100,256247,18.005,0.140278
4,MV001941940000,Across the Universe,2007,"[Musical, Romance]",82,53,2020-07-10T11:01:00+00:00,2021-01-01T12:59:00+00:00,FXM_HD,8.625,100,378921,8.625,304.601111
5,MV000317690000,Ghost,1990,"[Fantasy, Romance]",80,74,2019-03-07T05:00:00+00:00,2030-01-01T04:59:59+00:00,IND_HD_EXP,21.902,100,431391,21.902,2562.425000
6,MV000355470000,The Last of the Mohicans,1992,"[Historical drama, Adventure, War, Romance]",88,93,2020-11-11T05:00:00+00:00,2021-01-01T04:59:59+00:00,SHOWTIME_HD_EXP,15.792,100,611755,15.792,1825.788333
7,MV000193150000,Out of Africa,1985,"[Romance, Biography, Drama]",83,59,2019-03-21T04:00:00+00:00,2021-01-01T04:59:59+00:00,IND_HD_EXP,11.345,100,735447,11.345,75.934444
8,MV010308720000,Angry Angel,2017,"[Holiday, Romance, Fantasy]",83,NaN,2020-11-01T04:00:00+00:00,2020-12-26T04:59:00+00:00,FREEFORM_HD,3.646,100,374779,3.646,270.018333
9,MV010413570000,Aladdin,2019,"[Children, Musical comedy, Fantasy, Adventure,...",94,57,2020-05-01T04:00:00+00:00,2021-04-01T03:59:59+00:00,IND_HD_EXP,144.303,100,153207,144.303,16.943333


In [102]:
import plotly.express as px

fig = px.scatter(total_results, x= "OMDB_pop_score", y="audience_score", size="vt_hours",
            hover_name = "title", color="vt_hours", log_x=True)
           
fig.show()